In [56]:
import cv2
import numpy as np
import imutils
import datetime

class Panorama:

    # the blend types
    NO_BLEND = 1
    BOTH = 2
    
    # padding the images so we don't lose any information
    @classmethod
    def padblack(cls, left, right, increase=2):
        h = (left.shape[0] + right.shape[0])*increase
        w = (right.shape[1]+left.shape[1])*increase
        leftpadded = np.zeros([h, w ,3],dtype=np.uint8)
        rightpadded = np.zeros([h, w ,3],dtype=np.uint8)
        leftbeg = (w-left.shape[1])//2
        upbeg = (h-left.shape[0])//2
        leftpadded[upbeg:upbeg+left.shape[0], leftbeg:leftbeg+left.shape[1]] = left
        rightpadded[upbeg:upbeg+right.shape[0], leftbeg:leftbeg+right.shape[1]] = right
        return (leftpadded, rightpadded)
        
    # the new blending
    # using slicing instead of for loops, it's much faster
    # we make the weighed average of the two images
    # the idea roughly comes from here: http://bigwww.epfl.ch/publications/thevenaz0701.pdf
    # the above link found here: https://stackoverflow.com/questions/36386968/image-stitching-methods-to-remove-seams-for-stitched-image
    @classmethod
    def blendWeighed(cls, imgA, imgB, threshold=3):
        distA = cv2.distanceTransform(cv2.cvtColor(imgA, cv2.COLOR_BGR2GRAY), cv2.DIST_L2, 5)
        distB = cv2.distanceTransform(cv2.cvtColor(imgB, cv2.COLOR_BGR2GRAY), cv2.DIST_L2, 5)
        result = np.zeros(imgA.shape)
        # I don't know better atm but the dimensions of the distA, distB and the imgA and imgB are not fitting
        # and I cannot use slicing if they don't fit
        a = np.asarray(np.dstack((distA, distA, distA)), dtype=np.float32)
        b = np.asarray(np.dstack((distB, distB, distB)), dtype=np.float32)
        div = a + b
        mask = (imgA >= threshold) & (imgB >= threshold)
        result[mask] = (a*imgA+b*imgB)[mask]/div[mask]
        result[imgA < threshold] = imgB[imgA < threshold]
        result[imgB < threshold] = imgA[imgB < threshold]
        return result
    
    # obtaining the keypoints and their features for an image
    @classmethod
    def detectAndDescribe(cls, image):
        # convert the image to grayscale
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # detect and extract features from the image
        descriptor = cv2.xfeatures2d.SIFT_create()
        (kps, features) = descriptor.detectAndCompute(image, None)
        return (kps, features)
    
    # the method for drawing matches in the image left and right, given the 
    # keypoints and matches
    @classmethod
    def drawMatches(cls, right, kpsR, left, kpsL, matches):
        matchImg = None
        matchImg = cv2.drawMatches(right, kpsR, left, kpsL, matches, matchImg)
        return matchImg
    @classmethod
    def drawKeypoints(cls, image, keypoints):
        draw = None
        draw = cv2.drawKeypoints(image, keypoints, draw)
        return draw
    
    @classmethod
    def transform(cls, matches, kpsR, kpsL, left, right):
        matchesIdx = [(m.queryIdx, m.trainIdx) for m in matches[:10]]
        matchesCoordR = np.float32([kpsR[mind[0]].pt for mind in matchesIdx])
        matchesCoordL = np.float32([kpsL[mind[1]].pt for mind in matchesIdx])
        (H, mask) = cv2.findHomography(matchesCoordR, matchesCoordL, cv2.RANSAC)
        
        imgDest = None
        imgDest = cv2.warpPerspective(right, H, (right.shape[1], right.shape[0]))
        
        return imgDest
        
    @classmethod
    def makePanorama(cls, right, left, drawKeypoints=False, drawMatches=False, blend_type=NO_BLEND):
        # the result is a dictionary that contains all the possible images derived:
        # result with no blending
        # result with blending
        # right image with keypoints drawn
        # left image with keypoints drawn
        # both images with matches drawn
        result = {'resNoBlend':None, 'resBlend':None, 'kpsR':None, 'kpsL':None, 'matches':None}
        beg = datetime.datetime.now()
        
        l, r = cls.padblack(left, right)
        left, right = l, r
        
        # first, we find the keypoints in the images and their features 
        (kpsR, ftsR) = cls.detectAndDescribe(right)
        (kpsL, ftsL) = cls.detectAndDescribe(left)
        end = datetime.datetime.now()
        print("Keypoint detection in milliseconds: ", (end-beg).total_seconds()*1000)
        # if we should draw the keypoints, we draw them and put in the dictionary
        if drawKeypoints:
            result['kpsR'] = cls.drawKeypoints(right, kpsR)
            result['kpsL'] = cls.drawKeypoints(left, kpsL)
        
        
        
        #orb = cv2.ORB_create()
        
        # we find the matcher for the images and sort them by distance (by relevance)
        beg = datetime.datetime.now()
        
        bf = cv2.BFMatcher(cv2.NORM_L1,crossCheck=False)
        matches = bf.match(ftsR,ftsL)
        matches = sorted(matches, key=lambda x:x.distance)
        
        end = datetime.datetime.now()
        print("Matching in milliseconds: ", (end-beg).total_seconds()*1000)
        
        # if we should draw the matches, we draw them and put the result in the dictionary
        if drawMatches:
            result['matches'] = cls.drawMatches(right, kpsR, left, kpsL, matches[:10])
        
        beg = datetime.datetime.now()
        
        # we transform the right image
        newDst = cls.transform(matches, kpsR, kpsL, left, right)
        #cv2.imwrite('newDst.png', newDst)
       
        end = datetime.datetime.now()
        print("Applying the transformation in milliseconds: ", (end-beg).total_seconds()*1000)
    
        # we pad the left image with black background so that the two images have matching dimensions
        #newSrc = np.zeros([left.shape[0] + right.shape[0], right.shape[1]+left.shape[1] ,3],dtype=np.uint8)
        #newSrc[0:left.shape[0], 0:left.shape[1]] = left
        newSrc = left
        #cv2.imwrite('newSrc.png', newSrc)
        
        # now, both images that we use are in newDst (right) and newSrc (left)
        
        # depending on the type of blending chosen, we blend (or don't) and put the results 
        # in the dictionary
        # since the newSrc is now padded, we use left!
        beg = datetime.datetime.now()
        
        tmp_no_blend = cls.mergeProto(newDst, newSrc)
        cropped, (y, yh, x, xw) = cls.crop_rect(tmp_no_blend)
        result['resNoBlend'] = cropped
        newDst1 = newDst[y:yh, x:xw]
        newSrc1 = newSrc[y:yh, x:xw]
    
        end = datetime.datetime.now()
        print("Copying in milliseconds: ", (end-beg).total_seconds()*1000)
        
        if blend_type == cls.BOTH:
            beg = datetime.datetime.now()
            
            result['resBlend'] = cls.blendWeighed(newDst1, newSrc1)
            
            end = datetime.datetime.now()
            print("Blending in milliseconds: ", (end-beg).total_seconds()*1000)
        
        return result
    # cropping doesn't remove ALL the black parts, just the maximal amount that
    # keeps all the pixels of the stitched images 
    @classmethod
    def crop_new(cls,img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
        im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        x,y,w,h = cv2.boundingRect(cnt)
        cntarea = w*h
        # not sure if already sorted so finding maximal area rectangle, that one's ours
        for i in range(1, len(contours)):
            x,y,w,h = cv2.boundingRect(contours[i])
            newarea = w*h
            if newarea > cntarea:
                cnt = contours[i]
                cntarea = newarea
        x,y,w,h = cv2.boundingRect(cnt)
        crop = img[y:y+h,x:x+w]
        return crop
    
    @classmethod
    def crop_rect(cls, img):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        _,thresh = cv2.threshold(gray,1,255,cv2.THRESH_BINARY)
        im2, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
        cnt = contours[0]
        x,y,w,h = cv2.boundingRect(cnt)
        cntarea = w*h
        # not sure if already sorted so finding maximal area rectangle, that one's ours
        for i in range(1, len(contours)):
            x,y,w,h = cv2.boundingRect(contours[i])
            newarea = w*h
            if newarea > cntarea:
                cnt = contours[i]
                cntarea = newarea
        x,y,w,h = cv2.boundingRect(cnt)
        crop = img[y:y+h,x:x+w]
        return crop, (y, y+h, x, x+w)
    
    @classmethod
    def mergeProto(cls, im1, im2, threshold=10):
        im2cpy = im2.copy()
        # this gives me unspeakable pleasure
        mask = (im2cpy < threshold) & (im1 >= threshold)
        im2cpy[mask] = im1[mask]
        return im2cpy
        

In [61]:
beg = datetime.datetime.now()

right = cv2.imread('carmel-00.png')
left = cv2.imread('carmel-01.png')
# resizing, so it's faster
#right = imutils.resize(right, width=600)
#left = imutils.resize(left, width=600)
#l, r = Panorama.padblack(left, right)
res = Panorama.makePanorama(right, left, drawKeypoints=True, drawMatches=True, blend_type=Panorama.BOTH)

end = datetime.datetime.now()

print("Total time: ", (end-beg).total_seconds(), "s")

Keypoint detection in milliseconds:  9174.051000000001
Matching in milliseconds:  1200.494
Applying the transformation in milliseconds:  71.291
Copying in milliseconds:  117.982
Blending in milliseconds:  95.165
Total time:  10.832165 s


In [62]:
cv2.imwrite('demo/resBlendCarmel.png', res['resBlend'])
cv2.imwrite('demo/resNoBlendCarmel.png', res['resNoBlend'])
#TODO time all the parts and try to speed this up

True